## Image classification on DL (MobileNetV2). Face mask detection. Predictions at work Vol.2

### (Diploma project by student @Pawel_MTW "Skillfactory.ru")

### В Vol.1 мы построили модель для детектирования маски на лице человека,обучили ее, проверили показатели эффективности, сохранили ее.  Настало время внедрить модель в практическое использование. Этим мы и займемся. 

## Импорт библиотек/модулей. Основные настройки.

In [1]:
!pip install --upgrade pip
!pip install cvlib
!pip install opencv-python
!pip install xmltodict
!pip install mtcnn 
!pip install plot-metric
#!pip install face_recognition
!pip install --upgrade imutils

     |████████████████████████████████| 1.5 MB 1.3 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.3.3
    Uninstalling pip-20.3.3:
      Successfully uninstalled pip-20.3.3
     |████████████████████████████████| 10.0 MB 178 kB/s eta 0:00:01
  Created wheel for cvlib: filename=cvlib-0.2.6-py3-none-any.whl size=10044621 sha256=1d8972d8b7002234a08c50943372d5ea33dd813d235bc1927ef4b952229fcbea
  Stored in directory: /root/.cache/pip/wheels/9b/96/9b/373c07517ffe0bedbebc0813aec1e62cb2bb1dee91b3694895
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=230f134b4565bffb726d47fe9efc30370bb27b4ae74487ba9be7768a869b3cfb
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12075 sha256=39d339c02da8e6910f85849cedd503e31f0dfe12ac8f381c9d8c15cde1524f36
  Stored in directory: /root/.cache/pip/wheel

In [2]:
import cv2
import json
import os
import argparse
import xmltodict
import datetime
import plot_metric
import pylab as pl
import cvlib as cv
import numpy as np 
import pandas as pd 
import seaborn as sns
import sys, cv2, time
from PIL import Image
import tensorflow as tf
import scikitplot as skplt
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot
import matplotlib.pyplot as plt
from keras.utils import plot_model
from scipy.spatial import distance
from keras.models import Sequential
from warnings import filterwarnings
from keras.models import load_model
from matplotlib.patches import Rectangle
from keras.optimizers import Adam, RMSprop
from sklearn.metrics import roc_auc_score
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from plot_metric.functions import BinaryClassification
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Flatten
from keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import SpatialDropout2D, BatchNormalization, Input, Activation
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint


IMG_SIZE = 124 # какого размера подаем изображения в сеть
IMG_CHANNELS = 3   # у RGB 3 канала
input_shape  = [IMG_SIZE, IMG_SIZE, IMG_CHANNELS]

batch_size = 32
MIN_DISTANCE = 80

num_classes = 2
labels=["No Mask","Mask"]
mask_label = {0:"No Mask",1:"Mask"}
color_label = {0:(0,0,255),1:(0,255,0),2:(255,0,0),3:(0,0,0),4:(255,255,255)}
dist_label = {0:(0,255,0),1:(255,0,0)}

DATA_PATH = '../input/'
PATH = "../working/" 
face_model = cv2.CascadeClassifier('../input/haar-cascades-for-face-detection/haarcascade_frontalface_default.xml')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    pass   # print(dirname)
    #print(os.path.join(dirname, filename))
    
#print(os.listdir(DATA_PATH))from keras import models 

## Load model. Загружаем обученную модель. 

In [3]:
#import tensorflow as tf№
model = tf.keras.models.load_model("../input/readymodel3/model_mask (2).hdf5",
                                  custom_objects=None, compile=True, options=None)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.40_224 (Functi (None, 4, 4, 1792)        4363712   
_________________________________________________________________
global_max_pooling2d (Global (None, 1792)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               459008    
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_6 (Activation)    (None, 1)                

### Мы обучали свою модель на картинках, где лицо было в рамках картинки, поэтому, если мы хотим  сделать свою модель применимой к любому изображению человека, нам нужно решить проблему детектирования лица.

In [4]:
input_shape  = [IMG_SIZE, IMG_SIZE, IMG_CHANNELS]
input_shape

[120, 120, 3]

In [5]:
color_label = {0:(0,0,255),1:(0,255,0),2:(255,0,0)}
image = cv2.imread('../input/faces-with-masks/faces_with_mask/1006.jpg')
gray = cv2.cvtColor(image, cv2.IMREAD_GRAYSCALE)
faces = face_model.detectMultiScale(gray,1.1,4) 
out_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 

for i in range(len(faces)):
    (x,y,w,h) = faces[i]
    crop = out_img[y:y+h,x:x+w]
    crop = cv2.resize(crop,(IMG_SIZE,IMG_SIZE))
    crop = np.reshape(crop,[1,IMG_SIZE,IMG_SIZE,IMG_CHANNELS])/255.0
    mask_result = model.predict(crop)
    if mask_result>0.5:
        label_Y = 1
    else:
        label_Y = 0
    cv2.rectangle(out_img,(x,y),(x+w,y+h),(color_label[0]),2)
    cv2.putText(out_img,mask_label[label_Y] , (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (color_label[1]), 2)
plt.figure(figsize=(10,10))
plt.imshow(out_img)

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1461 run_step  **
        outputs = mo

### Для решения этой проблемы есть много разных вариантов. Мы начнем с самого простого и нересурсоемкого: OpenCV haar cascade.

### Он неплохо справляется с фронтальными лицами (есть разные модули).

### Создаем алгоритм, который находит лица на картинке и отмечаем их синим квадратом.

In [ ]:

image = cv2.imread('../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/0011.jpg')
gray = cv2.cvtColor(image, cv2.IMREAD_GRAYSCALE)
face_model = cv2.CascadeClassifier('../input/haar-cascades-for-face-detection/haarcascade_frontalface_default.xml')
faces = face_model.detectMultiScale(gray) #returns a list of (x,y,w,h) tuples
out_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #colored output image

#plotting
for (x,y,w,h) in faces:
    cv2.rectangle(out_img,(x,y),(x+w,y+h),(color_label[0]), 2)   # blue  
plt.figure(figsize=(10,10))
plt.imshow(out_img)

### Добавим рассчет расстояния между людьми на фото (MIN_DISTANCE) и оценку соблюдения этого условия: если дистанция соблюдена -  рамка зеленая, если нет - красная.Если лиц меньше 2, то выводится сообщение, что мало лиц.


In [ ]:
MIN_DISTANCE = 220

if len(faces)>=2:
    label = [0 for i in range(len(faces))]
    for i in range(len(faces)-1):
        for j in range(i+1, len(faces)):
            dist = distance.euclidean(faces[i][:2],faces[j][:2])
            if dist<MIN_DISTANCE:
                label[i] = 1
                label[j] = 1

    for i in range(len(faces)):
        (x,y,w,h) = faces[i]
        if label[i]==1:
            cv2.rectangle(out_img,(x,y),(x+w,y+h),(color_label[2]),2) # red - if bad
        else:
            cv2.rectangle(out_img,(x,y),(x+w,y+h),(color_label[1]),2) #green - if good
    plt.figure(figsize=(10,10))
    plt.imshow(out_img)
            
else:
    print("No. of faces detected is less than 2")

### Добавляем нашу обученную модель и predictions:

In [ ]:
if len(faces)>=2:
    label = [0 for i in range(len(faces))]
    for i in range(len(faces)-1):
        for j in range(i+1, len(faces)):
            dist = distance.euclidean(faces[i][:2],faces[j][:2])
            if dist<MIN_DISTANCE:
                label[i] = 1
                label[j] = 1

    for i in range(len(faces)):
        (x,y,w,h) = faces[i]
        crop = out_img[y:y+h,x:x+w]
        crop = cv2.resize(crop,(IMG_SIZE,IMG_SIZE))
        crop = np.reshape(crop,[1,IMG_SIZE,IMG_SIZE,IMG_CHANNELS])/255.0
        mask_result = model.predict(crop)                     
        if mask_result > 0.5:
            label_Y = 1
        if mask_result < 0.5:
            label_Y = 0
        cv2.putText(out_img,mask_label[(label_Y)] , (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,0.5,dist_label[label[i]],2)
        cv2.rectangle(out_img,(x,y),(x+w,y+h),dist_label[label[i]],2)
    plt.figure(figsize = (10,10))
    plt.imshow(out_img)
            
else:
    print("No. of faces detected is less than 2")

In [ ]:
mask_result

In [ ]:
print(len(faces))

### Проблема: не всегда Open CV haarCascade точно определяет все лица на фото (точность -+80%)

In [ ]:
image = cv2.imread('../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/0051.jpg')
gray = cv2.cvtColor(image, cv2.IMREAD_GRAYSCALE)
face_model = cv2.CascadeClassifier('../input/haar-cascades-for-face-detection/haarcascade_frontalface_default.xml')
faces = face_model.detectMultiScale(gray) 
out_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 

if len(faces)>=2:
    label = [0 for i in range(len(faces))]
    for i in range(len(faces)-1):
        for j in range(i+1, len(faces)):
            dist = distance.euclidean(faces[i][:2],faces[j][:2])
            if dist<MIN_DISTANCE:
                label[i] = 1
                label[j] = 1
                
    for i in range(len(faces)):
        (x,y,w,h) = faces[i]
        crop = out_img[y:y+h,x:x+w]
        crop = cv2.resize(crop,(IMG_SIZE,IMG_SIZE))
        crop = np.reshape(crop,[1,IMG_SIZE,IMG_SIZE,IMG_CHANNELS])/255.0
        mask_result = model.predict(crop) 
        
        if mask_result > 0.5:
            label_Y = 1
        if mask_result < 0.5:
            label_Y = 0
        cv2.putText(out_img, mask_label[(label_Y)] , (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,0.5,dist_label[label[i]],2) 
        cv2.rectangle(out_img,(x,y),(x+w,y+h),dist_label[label[i]],2)
        
    plt.figure(figsize=(10,10))
    plt.imshow(out_img) 
    
else:
    print("No. of faces detected is less than 2")

### На фото толпы проблема детекции лица стоит особо остро у haar cascade:

In [ ]:
image = cv2.imread("../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/0025.jpg")
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) 
faces = face_model.detectMultiScale(gray) 
out_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 

for (x,y,w,h) in faces:
    cv2.rectangle(out_img,(x,y),(x+w,y+h),(0,0,255),2)
plt.figure(figsize=(8,8))
plt.imshow(out_img)

### Попробуем  вариант с MTCNN (предобученной нейронной сетью) для детектирования лиц.

### MTCNN очень полезна, так как может работать в режиме реального времени даже на небольших устройствах. После MTCNN было много алгоритмов,но он остается одним из лучших для обнаружения лиц (frontalface  лучше всего), с гораздо лучшей точностью, чем  Open CV (haar cascade).

In [ ]:
# draw an image with detected objects
def draw_image_with_boxes(img, result_list):
    # load the image
    data = pyplot.imread(img)
    # plot the image
    pyplot.imshow(data)
    # get the context for drawing boxes
    ax = pyplot.gca()
    # plot each box
    for result in result_list:
        # get coordinates
        x, y, width, height = result['box']
        # create the shape
        rect = Rectangle((x, y), width, height, fill=False, color='blue')
        # draw the box
        ax.add_patch(rect)
    # show the plot
    pyplot.show()

In [ ]:
img = '../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/0025.jpg'
image = pyplot.imread(img)
detector = MTCNN()
# detect faces in the image
faces = detector.detect_faces(image)
# display faces on the original image
draw_image_with_boxes(img, faces)

In [ ]:
# вот такую информацию MTCNN выводит по каждому лицу, достаточно подробно:
faces

#### 'box' - выводит координаты лица, 'confidence' - уверенность в том, что это лицо.

### Как видим, результат детекции лиц гораздо лучше!

In [ ]:
img = '../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/0044.jpg'
# load image from file
image = pyplot.imread(img)
# create the detector, using default weights
detector = MTCNN()
# detect faces in the image
faces = detector.detect_faces(image)
draw_image_with_boxes(img, faces)

### А вот так MTCNN работает в связке с нашей моделью: сперва детектируются лица, затем измеряется расстояние между лицами (если Min расстояние не соблюдено - то рамки таких лиц красные, если Norm - то зеленые), затем анализируется каждое лицо через нашу обученную модель на предмет наличия маски на лице.

In [ ]:
MIN_DISTANCE = 70
image = pyplot.imread('../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/0051.jpg')
detector = MTCNN()
faces = detector.detect_faces(image)
out_img = image 

if len(faces)>=2:
    label = [0 for i in range(len(faces))]
    for i in range(len(faces)-1):
        for j in range(i+1, len(faces)):
            dist = distance.euclidean(faces[i]["box"][:2],faces[j]["box"][:2])
            if dist<MIN_DISTANCE:
                label[i] = 1
                label[j] = 1

    for i in range(len(faces)):
        (x,y,w,h) = faces[i]["box"]
        crop = out_img[y:y+h,x:x+w]
        crop = cv2.resize(crop,(IMG_SIZE,IMG_SIZE))
        crop = np.reshape(crop,[1,IMG_SIZE,IMG_SIZE,IMG_CHANNELS])/255.0
        mask_result = model.predict(crop)        
        if mask_result > 0.5:
            label_Y = 1
        elif mask_result < 0.5:
            label_Y = 0
        cv2.putText(out_img,mask_label[(label_Y)] , (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,0.5,dist_label[label[i]],2)
        cv2.rectangle(out_img,(x,y),(x+w,y+h),dist_label[label[i]],2)        
else:
    print("No. of faces detected is less than 2")
plt.figure(figsize=(12,12))
plt.imshow(out_img)

In [ ]:
faces[2]["box"] #координаты лица №2. Такие координаты мы и будем в дальнейшем использовать.

#### Другой вариант оформления вывода информации после анализа:

In [ ]:
image = pyplot.imread('../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/0051.jpg')
detector = MTCNN()

faces = detector.detect_faces(image)
out_img = image
plt.figure(figsize=(15,15)) 

if len(faces)>=2:
    label = [0 for i in range(len(faces))]
    for i in range(len(faces)-1):
        for j in range(i+1, len(faces)):
            dist = distance.euclidean(faces[i]["box"][:2],faces[j]["box"][:2])
            if dist<MIN_DISTANCE:
                label[i] = 1
                label[j] = 1    

    for i in range(len(faces)):
        (x,y,w,h) = faces[i]["box"]
        crop = out_img[y:y+h,x:x+w]
        crop = cv2.resize(crop,(IMG_SIZE,IMG_SIZE))
        crop = np.reshape(crop,[1,IMG_SIZE,IMG_SIZE,IMG_CHANNELS])/255.0
        mask_result = model.predict(crop) 
      
        if mask_result > 0.5:
            label_Y = 1
        elif mask_result < 0.5:
            label_Y = 0        
        cv2.rectangle(out_img,(x,y),(x+w,y+h),dist_label[label[i]],3) 
        cv2.rectangle(out_img,(x,y-30),(x+w,y),dist_label[label[i]],-1)
        cv2.putText(out_img, mask_label[(label_Y)], (x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),2)
else:
    print("No. of faces detected is less than 2")            
plt.imshow(out_img)  

### Изменим  код, чтобы он был применим и для 1 человека в кадре.

In [ ]:
filename = '../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/0015.jpg'
image = pyplot.imread(filename)
detector = MTCNN()
faces = detector.detect_faces(image)
out_img = image

if len(faces)>=2:
    label = [0 for i in range(len(faces))]
    for i in range(len(faces)-1):
        for j in range(i+1, len(faces)):
            dist = distance.euclidean(faces[i]["box"][:2],faces[j]["box"][:2])
            if dist<MIN_DISTANCE:
                label[i] = 1
                label[j] = 1    

    for i in range(len(faces)):
        (x,y,w,h) = faces[i]["box"]
        crop = out_img[y:y+h,x:x+w]
        crop = cv2.resize(crop,(IMG_SIZE,IMG_SIZE))
        crop = np.reshape(crop,[1,IMG_SIZE,IMG_SIZE,IMG_CHANNELS])/255.0
        mask_result = model.predict(crop) 
                          
        if mask_result > 0.5:
            label_Y = 1
        elif mask_result < 0.5:
            label_Y = 0        
        cv2.rectangle(out_img,(x,y),(x+w,y+h),dist_label[label[i]],3) 
        cv2.rectangle(out_img,(x,y-30),(x+w,y),dist_label[label[i]],-1)
        cv2.putText(out_img, mask_label[(label_Y)], (x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),2)

else:
    for i in range(len(faces)):
        (x,y,w,h) = faces[i]["box"]
        crop = out_img[y:y+h,x:x+w]
        crop = cv2.resize(crop,(IMG_SIZE,IMG_SIZE))
        crop = np.reshape(crop,[1,IMG_SIZE,IMG_SIZE,IMG_CHANNELS])/255.0
        mask_result = model.predict(crop) 
         
        if mask_result > 0.55:
            cv2.rectangle(out_img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.rectangle(out_img,(x,y-50),(x+w,y),(0,255,0),-1)
            cv2.putText(out_img,mask_label[1],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)     
        if mask_result < 0.45:
            cv2.rectangle(out_img,(x,y),(x+w,y+h),(255,0,0),3)
            cv2.rectangle(out_img,(x,y-50),(x+w,y),(255,0,0),-1)
            cv2.putText(out_img,mask_label[0],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)     
        
pyplot.imshow(out_img) 

In [ ]:
 pyplot.imshow(out_img[y:y+h,x:x+w]) 

### Создадим финальный код для работы с фото для вывода всей нужной нам информации: время, кол-во лиц в кадре, предупреждение, если людей больше 3+ (Х человек), анализ лиц на наличие масок, а также анализ на соблюдение социальной дистанции между людьми:

In [ ]:
image = pyplot.imread('../input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images/0007.jpg')
detector = MTCNN()
faces = detector.detect_faces(image)
out_img = image
plt.figure(figsize=(15,15)) 
time_now = datetime.datetime.now().replace(microsecond=0)
cv2.putText(out_img,str(time_now),(100,70),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
cv2.putText(out_img, f"People: {len(faces)}",(100,110),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)  

if len(faces)>=2:
    label = [0 for i in range(len(faces))]
    for i in range(len(faces)-1):
        for j in range(i+1, len(faces)):
            dist = distance.euclidean(faces[i]["box"][:2],faces[j]["box"][:2])
            if dist<MIN_DISTANCE:
                label[i] = 1
                label[j] = 1    

    for i in range(len(faces)):
        (x,y,w,h) = faces[i]["box"]
        crop = out_img[y:y+h,x:x+w]
        crop = cv2.resize(crop,(IMG_SIZE,IMG_SIZE))
        crop = np.reshape(crop,[1,IMG_SIZE,IMG_SIZE,IMG_CHANNELS])/255.0
        mask_result = model.predict(crop) 
                          
        if mask_result > 0.5:
            label_Y = 1
        elif mask_result < 0.5:
            label_Y = 0        
        cv2.rectangle(out_img,(x,y),(x+w,y+h),dist_label[label[i]],3) 
        cv2.rectangle(out_img,(x,y-30),(x+w,y),dist_label[label[i]],-1)
        cv2.putText(out_img, mask_label[(label_Y)], (x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),2)           

else:
    for i in range(len(faces)):
        (x,y,w,h) = faces[i]["box"]
        crop = out_img[y:y+h,x:x+w]
        crop = cv2.resize(crop,(IMG_SIZE,IMG_SIZE))
        crop = np.reshape(crop,[1,IMG_SIZE,IMG_SIZE,IMG_CHANNELS])/255.0
        mask_result = model.predict(crop) 
         
        if mask_result > 0.55:
            cv2.rectangle(out_img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.rectangle(out_img,(x,y-50),(x+w,y),(0,255,0),-1)
            cv2.putText(out_img,mask_label[1],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)     
        if mask_result < 0.45:
            cv2.rectangle(out_img,(x,y),(x+w,y+h),(255,0,0),3)
            cv2.rectangle(out_img,(x,y-50),(x+w,y),(255,0,0),-1)
            cv2.putText(out_img,mask_label[0],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)      

# Display alarm if number of people 3+:
if len(faces)>=3:
    cv2.putText(out_img,str("Alarm: 3+ people"),(100,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)           
            
plt.imshow(out_img) 

### В vol.3 мы продолжим усовершенствовать наш алгоритм для работы с видео потоком.